# Detección de elementos
En este ejemplo vamos a usa el modelo [YOLO](https://arxiv.org/pdf/1612.08242.pdf), gracias al cual podremos detectar los elementos que aparecen dentro de una fotgrafia o video.

La identificación que obtendremos con YOLO consite en una etiqueta indicando que es lo que se ha detectado y las posiciones en la que se enuentra este objeto.

La implementación que usaresmos será la proporcionado por la biblioteca OpenCV.

## Carga del modelos
A continuación realizamos la carga del modelo preentrenado par la ejecución de YOLO. 

Se le debe indicar el fichero de clases (que será lo que se podrá detectar), los pesos preentenados y el fichero de configuración del modelo.

In [1]:
import cv2
import numpy as np

_fichero_clases = './recursos_yolo/yolov3.txt'
_fichero_pesos = './recursos_yolo/yolov3.weights'
_fichero_configuracion = 'recursos_yolo/yolov3.cfg' 

# Lectura de las clases
classes = None
with open(_fichero_clases, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    
# Carga del modelo
net = cv2.dnn.readNet(_fichero_pesos, _fichero_configuracion)


## Carga de la imagen

Realizamos la carga de la imagen que queremso procesar, asi como su conversión a matriz y su preprocesmainto:escalado, normalizado de colores, etc. para que funcione de manera correcta el modelo

In [2]:
_path_imagen = './entradas/ej_2.jpg' 

image = cv2.imread(_path_imagen)

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

# Convierte la imagen en una matriz normalizada 
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

print(blob)

[[[[0.99176    0.99176    0.98784    ... 0.97608    0.99176
    0.99568003]
   [0.98784    0.94472003 0.93296003 ... 0.83496    0.87808
    0.98784   ]
   [0.99568003 0.80752003 0.72912    ... 0.34104002 0.49392
    0.98392004]
   ...
   [0.98392004 0.47824    0.25872    ... 0.24696    0.46648002
    0.98392004]
   [0.99176    0.87808    0.8428     ... 0.83888    0.87808
    0.99568003]
   [0.99176    0.99176    0.97608    ... 0.97608    0.99176
    0.99176   ]]

  [[0.99176    0.99176    0.98784    ... 0.97608    0.99176
    0.99568003]
   [0.98784    0.95648    0.94472003 ... 0.83888    0.882
    0.98784   ]
   [0.9996     0.83104    0.76048    ... 0.3528     0.50176
    0.98392004]
   ...
   [0.98       0.46256    0.24304    ... 0.23912    0.46256
    0.98392004]
   [0.99176    0.87808    0.8428     ... 0.83888    0.87808
    0.99568003]
   [0.99176    0.99176    0.97608    ... 0.97608    0.99176
    0.99176   ]]

  [[0.99568003 0.98784    0.99176    ... 0.96824    0.98392004
    0.

## Pasamos la imagen por el modelo
Una vez que se tiene la imagen convertida en una matriz ya sería posible pasar esta por el modelo.

Como resultado de pasar la imagen por el modelo se obtendrán las capas de salida. Que en caso de yolo V3 son multiples capas, como se puede ver a continuación.

In [3]:
net.setInput(blob)


def obtener_capas_salida(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# ejecutamos el procesamiento del modelo
outs = net.forward(obtener_capas_salida(net))

print (outs)

[array([[0.03042415, 0.04341022, 0.39466417, ..., 0.        , 0.        ,
        0.        ],
       [0.04205536, 0.03739486, 0.2932723 , ..., 0.        , 0.        ,
        0.        ],
       [0.04267532, 0.04025999, 0.7159595 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.95474887, 0.9503714 , 0.4723646 , ..., 0.        , 0.        ,
        0.        ],
       [0.9630899 , 0.9643203 , 0.32435405, ..., 0.        , 0.        ,
        0.        ],
       [0.9651958 , 0.9659401 , 0.8611804 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.01565011, 0.02240092, 0.04353913, ..., 0.        , 0.        ,
        0.        ],
       [0.01370748, 0.01544142, 0.44774392, ..., 0.        , 0.        ,
        0.        ],
       [0.02114592, 0.01633698, 0.06623674, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9771715 , 0.9825373 , 0.03684205, ..., 0.        , 0.        ,
        0.        ],
       [0.9833511 

Recorremos los objetos detectados. Para cada uno ellos obtenemos su valor de cofianza y eliminamos aquellos que no suepren el umbral fijado.


In [4]:
_umbral = 0.5


class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > _umbral:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])
            
print (class_ids)

[17, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Pintado de las cajas de los objetos

Una vez qe si tiene los objetos que se han detectado podemos pintar la caja que contenga el objeto y ponerle la etiqueta detectada


In [5]:
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

#Función para pintar al caja
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

i = 0
for box in boxes:
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    
    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    i = i+1

   
cv2.imshow("object detection", image)
cv2.waitKey()    
cv2.imwrite("./salidas/object-detection.jpg", image)
cv2.destroyAllWindows()

## Refinado de las cajas
Procesamos las cajas, para evitar las duplicidades y quedarnos con las máximas

In [ ]:
_nms_threshold = 0.4
            
# non-max suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, _umbral, _nms_threshold)

for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    
    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

 
cv2.imshow("object detection2", image)
cv2.waitKey()
cv2.imwrite("./salidas/object-detection2.jpg", image)
cv2.destroyAllWindows()